# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [184]:
#Import pyspark:
from pyspark.sql import SparkSession

In [185]:
spark = SparkSession.builder.appName('Logistic').getOrCreate()
spark

In [186]:
df_cust = spark.read.csv("customer_churn.csv",header=True,inferSchema=True)
df_cust.show(2)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|   Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|Wilson PLC|    1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
only showing top 2 rows



In [187]:
#To check the number of records in dataframe:
df_cust.count()

900

In [188]:
#To check the number of columns in dataframe
len(df_cust.columns)

10

In [189]:
#To print the schema:
df_cust.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [190]:
#To print the columns:
df_cust.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [191]:
#To print thew five point summary:
df_cust.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [192]:
#To import vector assembler:
from pyspark.ml.feature import VectorAssembler 

In [193]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],
                           outputCol='features')

In [194]:
output = assembler.transform(df_cust)

In [195]:
output.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|            features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|      Phillip White|4

In [196]:
final_data = output.select('features','Churn')

In [197]:
final_data.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



# TRAIN TEST SPLIT

In [198]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [199]:
train_data.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[22.0,11254.38,1....|    0|
|[25.0,9672.03,0.0...|    0|
|[26.0,8787.39,1.0...|    1|
|[28.0,8670.98,0.0...|    0|
|[28.0,9090.43,1.0...|    0|
|[28.0,11128.95,1....|    0|
|[28.0,11204.23,0....|    0|
|[28.0,11245.38,0....|    0|
|[29.0,5900.78,1.0...|    0|
|[29.0,8688.17,1.0...|    1|
|[29.0,9378.24,0.0...|    0|
|[29.0,9617.59,0.0...|    0|
|[29.0,10203.18,1....|    0|
|[29.0,11274.46,1....|    0|
|[29.0,13255.05,1....|    0|
|[30.0,6744.87,0.0...|    0|
|[30.0,7960.64,1.0...|    1|
|[30.0,8677.28,1.0...|    0|
|[30.0,8874.83,0.0...|    0|
|[30.0,10744.14,1....|    1|
+--------------------+-----+
only showing top 20 rows



In [200]:
from pyspark.ml.classification import LogisticRegression

In [201]:
lr = LogisticRegression(labelCol='Churn')
model = lr.fit(train_data)
model_summ =model.summary


In [202]:
model_summ.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                613|                613|
|   mean|0.17128874388254486|0.12561174551386622|
| stddev|0.37706876397448996|0.33168192790642975|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [203]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [204]:
pred_and_labels = model.evaluate(test_data)

In [205]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.66253786966093...|[0.99872373183177...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.67407140659283...|[0.99657789486431...|       0.0|
|[29.0,12711.15,0....|    0|[5.77084736610217...|[0.99689257115611...|       0.0|
|[29.0,13240.01,1....|    0|[7.20393373663689...|[0.99925689790606...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.12530974896435...|[0.99781795806907...|       0.0|
|[30.0,10183.98,1....|    0|[2.86979792543371...|[0.94633308620263...|       0.0|
|[30.0,10960.52,1....|    0|[2.41529337845748...|[0.91798609056327...|       0.0|
|[30.0,12788.37,0....|    0|[2.45580899260248...|[0.92098521474778...|       0.0|
|[30.0,13473.35,0....|    0|[2.69742062475842...|[0.93687426922957...|       0.0|
|[31.0,5304.6,0.

In [206]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='Churn')

In [207]:
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

0.8155188246097337